<a href="https://colab.research.google.com/github/claudiabozza/ProgrammingAssignment2/blob/master/Kraken_api_cryptocurrencies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## How to pull all trade data using the Kraken REST API

How to pull all trade data using the Kraken REST API OHLC data does not go back beyond a certain number of frames and depending on the interval it won't go back past 24 hours. The "since" value is mainly used to receive incremental updates.

You can build your own OHLC from trades data. Start with "since" of 0 to get trade data from the beginning.

https://api.kraken.com/0/public/Trades?pair=XXBTZEUR&since=0

Then from the returned data, grab the "last" time ID (i.e. ...,"last":"1383839436659595694"}})

Use that ID for the "since" value in the new poll to get the next set of data:

https://api.kraken.com/0/public/Trades?pair=XXBTZEUR&since=1383839436659595694


# First load all libraries

In [0]:
!pip install retrying

  Running setup.py bdist_wheel for retrying ... - done
  Stored in directory: /root/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [0]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
import glob
import numpy as np
from retrying import retry
import re
import os

In [0]:
from google.colab import files

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# Functions to read Kraken API and save csv files to your Google Drive

### Chose parameters

In [0]:
currency = 'DASHUSD' # cryptocurrency code
since = 0 # initial timestamp
end_timestamp = '1541476117' # final timestamp

In [0]:
# chose where you're going to save your csv files, starting with '/content/gdrive/My Drive/', like the following example.
google_drive_path = '/content/gdrive/My Drive/crypto/dash/' 

### Load functions

In [0]:
def get_trade (pair = None, since=None):
    url = "https://api.kraken.com/0/public/Trades?pair=%s&since=%s" % (pair, since)
    r = requests.get(url)
    return r.json()

In [0]:
trade = get_trade(pair = currency, since = since)

In [0]:
print("last timestamp:", trade['result']['last'])
print("first line:", trade['result']['DASHUSD'][1])
print("first timestamp:", trade['result']['DASHUSD'][1][2])

last timestamp: 1492513658687884900
first line: ['70.000000', '0.61000000', 1492019142.6918, 'b', 'm', '']
first timestamp: 1492019142.6918


In [0]:
def get_all_trades (currency=None, since=None, end_timestamp=None, path=None):
    total_files = 0
    appended_data = []
    trade = get_trade(currency, since)
    end_timestamp = end_timestamp
    while trade['result']['last'] <= end_timestamp or len(trade['result'][currency]) == 0:
        trade = get_trade(currency, since)
        partial_df = pd.DataFrame(list(trade['result'][currency]), 
                                  columns = ['price', 'volume', 'timestamp', 'buy_sell', 'limit_or_market', 'x'])
        appended_data.append(partial_df)
        save_path = '%s/%s-%s-%s.csv' % (path, currency, since, trade['result']['last'])
        partial_df.to_csv(save_path)
        since = trade['result']['last']
        total_files = total_files + 1
        print("Total files:", total_files)
    df = pd.concat(appended_data)
    return df

In [0]:
all_trades = get_all_trades(currency=currency, since=since, end_timestamp=end_timestamp, path=google_drive_path)

###  In case of <font color='red'> error </font>  with previous function, retry loading the code below

In [0]:
@retry
def retry_if_error(path=None, currency=None, end_timestamp = None):
    try:
        list_of_files = glob.glob('%s/*' % path)
        latest_file = max(list_of_files, key=os.path.getctime)
        since = re.search('-([0-9]+).csv', latest_file, re.IGNORECASE).group(1)
        all_trades = get_all_trades(currency = currency, 
                                    since=since, 
                                    end_timestamp = end_timestamp, 
                                    path=path)
    except:
        pass
    return all_trades

In [0]:
new_files = retry_if_error(path=google_drive_path,
                          currency = currency,
                          end_timestamp = end_timestamp)

# Save all csv files from your Google Drive to an unique csv file to your computer

In [0]:
path = google_drive_path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
frame = pd.concat(list_)

In [0]:
# This is how your data looks like
frame = frame[['price', 'volume', 'timestamp','buy_sell']]
frame.head(1)

,price,volume,timestamp,buy_sell
0,66.85,3.6,1.492018e+09,b


### Chose a name for your unique csv file, like the following example

In [0]:
file_name = 'dash_data.csv'

In [0]:
frame.to_csv(file_name)
files.download(file_name)